In [2]:
# set up matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({"font.size": 14})

import numpy as np
import xarray as xr
from scipy.special import j0
from scipy.interpolate import RectBivariateSpline
from xbout import open_boutdataset

In [3]:
cases = ["sq_mid_nocut","sq_mid_nocut2","sq_mid_nocut3"]
grids = ["lowest", "low", "mid"]
data = []
ncs = []
for c, case in enumerate(cases):
    filepath = str(case) + "/BOUT.dmp.*.nc"

    ds = open_boutdataset(datapath=filepath, chunks={"t": 4})
    dsn = xr.open_dataset("sq_mid_nocut_" + str(grids[c]) + "res.fci.nc")

    # Use squeeze() to get rid of the y-dimension, which has length 1 usually unless turbulent.
    ds = ds.squeeze(drop=True)
    dsn = dsn.squeeze(drop=True)

    dx = ds["dx"].isel(z=0).values
    # Get rid of existing "x" coordinate, which is just the index values.
    #ds = ds.drop("x")
    #Create a new coordinate, which is length in units of rho_s
    ds = ds.assign_coords(x=np.arange(ds.sizes["x"])*dx)

    data.append(ds)
    ncs.append(dsn)

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 46MB
Dimensions:             (x: 68, y: 1, z: 68, t: 201)
Coordinates:
    dx                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dy                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dz                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 544B 0 1 2 3 4 5 6 7 ... 61 62 63 64 65 66 67
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 544B 0.007353 0.02206 ... 0.9779 0.9926
Data variables: (12/52)
    Bxy                 (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G1                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G2                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 172MB
Dimensions:             (x: 132, y: 1, z: 132, t: 201)
Coordinates:
    dx                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    dy                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    dz                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 1kB 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 1kB 0.003788 0.01136 ... 0.9886 0.9962
Data variables: (12/52)
    Bxy                 (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    G1                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    G2                  (x, y, z) float64 139kB dask.array<

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


In [4]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
def make_fluid_near_ghost_mask(inside, ghost):
    """
    Return a mask for fluid cells that have at least one ghost neighbor
    in +/-x or +/-z.
    """

    # Start with no neighbors marked
    nx, nz = inside.shape

    ghost_xp = np.zeros_like(ghost, dtype=bool)
    ghost_xm = np.zeros_like(ghost, dtype=bool)
    ghost_zp = np.zeros_like(ghost, dtype=bool)
    ghost_zm = np.zeros_like(ghost, dtype=bool)

    # Shift ghost mask to find neighbors
    ghost_xp[:-1, :] = ghost[1:, :]   # neighbor at +x
    ghost_xm[1:,  :] = ghost[:-1, :]  # neighbor at -x
    ghost_zp[:, :-1] = ghost[:, 1:]   # neighbor at +z
    ghost_zm[:, 1:]  = ghost[:, :-1]  # neighbor at -z

    has_ghost_neighbor = ghost_xp | ghost_xm | ghost_zp | ghost_zm

    # We only care about FLUID cells that see a ghost
    mask_fluid_near_ghost = inside.astype(bool) & has_ghost_neighbor.astype(bool)
    return mask_fluid_near_ghost

def l2_error_near_ghost_cells(dens, dens_true, inside, ghost, V=None, t_index=-1):
    """
    L2 error of density at time t_index, restricted to fluid cells
    that have at least one ghost neighbor in x or z.
    """
    if V is None:
        V = np.ones_like(inside, dtype=float)

    mask_edge = make_fluid_near_ghost_mask(inside, ghost)
    diff = dens[t_index] - dens_true[t_index]  # (nx, nz)
    l2_edge = np.sqrt(np.nanmean(diff.where(mask_edge)**2))
    
    xsz = np.shape(dens[t_index])[0]
    zsz = np.shape(dens[t_index])[1]
    xcut = xsz//2 - 2*xsz//64
    zcut = (zsz-4)//2 - 2*(zsz-4)//64
    ctr_diff = dens[1,xcut:-xcut,zcut:-zcut] - dens_true[1,xcut:-xcut,zcut:-zcut]
    l2_ctr = np.sqrt(np.nanmean(ctr_diff**2))
    return l2_edge, mask_edge, l2_ctr

def interpolate_on_boundary(x, z, density, dens_true, xb, zb, method="linear", t_index=-1):
    """
    Interpolate numerical and true density onto boundary points.

    Parameters
    ----------
    x, z : 1D arrays
        Grid coordinates in x and z directions.
    density, dens_true : 2D arrays, shape (len(x), len(z))
        Numerical and reference ("true") fields at a fixed time.
    xb, zb : arrays of same shape
        Boundary point coordinates (x_b, z_b).
    method : {"linear", "nearest"}, optional
        Interpolation method for RegularGridInterpolator.
        ("linear" ~ bilinear in 2D)

    Returns
    -------
    dens_b : array, same shape as xb
        Interpolated numerical density at boundary.
    dens_true_b : array, same shape as xb
        Interpolated true density at boundary.
    err_b : array, same shape as xb
        Pointwise error dens_b - dens_true_b.
    """
    # Flatten boundary points to (N, 2)
    pts = np.column_stack((xb.ravel(), zb.ravel()))

    interp_num = RegularGridInterpolator(
        (x, z), density[t_index,:,:].values,
        method=method,
        bounds_error=False,
        fill_value=None,  # extrapolate if slightly outside
    )
    interp_true = RegularGridInterpolator(
        (x, z), dens_true[t_index,:,:].values,
        method=method,
        bounds_error=False,
        fill_value=None,
    )

    dens_b_flat = interp_num(pts)
    dens_true_b_flat = interp_true(pts)

    dens_b = dens_b_flat.reshape(xb.shape)
    dens_true_b = dens_true_b_flat.reshape(xb.shape)
    err_b = dens_b - dens_true_b
    
    err_flat = err_b.ravel()
    l2 = np.sqrt(np.mean(err_flat**2))
    linf = np.max(np.abs(err_flat))
    return l2,linf


In [7]:
import numpy as np
from scipy.interpolate import RectBivariateSpline

def ib_boundary_errors_from_ghostid(
    dens_t,                 # 2D (nx,nz) at one timestep
    dens_true_t,            # Real dens for testing image points.
    ghost_id,               # 2D (nx,nz) ints: -1 or gid into *_pts arrays
    gst_pts, bdy_pts, img_pts,  # 2D (Ng,2): gst_pts are (ix,iz) indices; bdy/img are (R,Z)
    normals,                # 2D (Ng,2): (nR,nZ)
    norm_dist,              # 1D (Ng,)
    Rarr, Zarr,             # 2D (nx,nz) physical coordinates
    is_plasma,
    mask=None,              # optional 2D bool; if provided, reports stencil validity fraction
    bc_val_dir=0.0,         # constant Dirichlet value (ignored if phiB_true_fn provided)
    bc_val_neu=0.0,         # constant Neumann value (ignored if g_true_fn provided)
    phiB_true_fn=None,      # callable (R,Z)->phi_B_true for Dirichlet
    g_true_fn=None,         # callable (R,Z,nR,nZ)->dphi/dn true for Neumann
    use_norm_dist=True,     # if False, uses |I-B| computed from points
    check_equidistance=True):
    """
    Postprocess boundary errors for IB ghost-cell method using stored G,B,I pairing.

    Dirichlet:
      phi_B_hat = 0.5*(phi_G + phi_I)

    Neumann:
      g_hat = (phi_I - phi_G)/(2*d)  where d = norm_dist[gid] or |I-B|

    Returns a dict with L2/Linf metrics and some diagnostics.
    """

    dens_t = np.asarray(dens_t)
    ghost_id = np.asarray(ghost_id)

    # spline axes
    R1d = np.asarray(Rarr)[:, 0]
    Z1d = np.asarray(Zarr)[0, :]

    # bilinear spline for phi_I
    spl = RectBivariateSpline(R1d, Z1d, dens_t, kx=1, ky=1)
    spl_true = RectBivariateSpline(R1d, Z1d, dens_true_t, kx=1, ky=1)

    # find ghost grid locations and their ghost-record ids
    ii, kk = np.where(ghost_id >= 0)
    if ii.size == 0:
        return {
            "n_ghost": 0,
            "dir_l2": np.nan, "dir_linf": np.nan,
            "neu_l2": np.nan, "neu_linf": np.nan,
        }

    gid = ghost_id[ii, kk].astype(int)

    # phi_G from ghost grid cell
    phiG = dens_t[ii, kk]

    # geometry at ghost-record level
    BI = img_pts[gid]          # (R,Z)
    BB = bdy_pts[gid]          # (R,Z)
    nR = normals[gid, 0]
    nZ = normals[gid, 1]

    # phi_I from spline at image points
    phiI = spl.ev(BI[:, 0], BI[:, 1])
    phiI_true = spl_true.ev(BI[:,0], BI[:,1])

    # ---- Dirichlet error ----
    phiB_hat = 0.5 * (phiG + phiI)

    if phiB_true_fn is not None:
        phiB_true = phiB_true_fn(BB[:, 0], BB[:, 1])
    else:
        phiB_true = bc_val_dir * np.ones_like(phiB_hat)

    err_dir = phiB_hat - phiB_true
    dir_l2 = np.sqrt(np.mean(err_dir**2))
    dir_linf = np.max(np.abs(err_dir))

    # ---- Neumann error ----
    if use_norm_dist:
        d = np.asarray(norm_dist)[gid].astype(float)
    else:
        d = np.sqrt((BI[:, 0] - BB[:, 0])**2 + (BI[:, 1] - BB[:, 1])**2)

    d = np.where(d > 0, d, np.nan)
    g_hat = (phiI - phiG) / (2.0 * d)

    if g_true_fn is not None:
        g_true = g_true_fn(BB[:, 0], BB[:, 1], nR, nZ)
    else:
        g_true = bc_val_neu * np.ones_like(g_hat)

    err_neu = g_hat - g_true
    neu_l2 = np.sqrt(np.nanmean(err_neu**2))
    neu_linf = np.nanmax(np.abs(err_neu))

    err_img = phiI - phiI_true

    # how many of the 4 nodes are plasma for each gid
    n_plasma = np.sum(is_plasma.astype(np.int32), axis=1)  # 0..4
    m_all_plasma  = (n_plasma == 4)  # all 4 plasma
    m_one_ghost   = (n_plasma == 3)  # exactly 1 ghost/non-plasma
    m_multi_ghost = (n_plasma <= 2)  # 2+ ghost/non-plasma

    def masked_norms(err, mask):
        # mask out everything else as NaN so nanmean/nanmax ignore them
        e = np.where(mask, err, np.nan)
        l2 = np.sqrt(np.nanmean(e**2))
        linf = np.nanmax(np.abs(e))
        return l2, linf

    img_l2_all,   img_linf_all   = masked_norms(err_img, m_all_plasma)
    img_l2_one,   img_linf_one   = masked_norms(err_img, m_one_ghost)
    img_l2_multi, img_linf_multi = masked_norms(err_img, m_multi_ghost)
    img_l2 = np.sqrt(np.nanmean(err_img**2))
    img_linf = np.nanmax(np.abs(err_img))

    out = {
        "n_ghost": int(ii.size),
        "dir_l2": float(dir_l2),
        "dir_linf": float(dir_linf),
        "neu_l2": float(neu_l2),
        "neu_linf": float(neu_linf),
        "img_l2": float(img_l2_all),
        "img_linf": float(img_linf_all)
    }

    # ---- Optional diagnostics ----
    if check_equidistance:
        # Map gst_pts[gid]=(ix,iz) to physical (R,Z)
        gix = np.asarray(gst_pts)[gid, 0].astype(int)
        giz = np.asarray(gst_pts)[gid, 1].astype(int)
        nx, nz = dens_t.shape
        gix = np.clip(gix, 0, nx - 1)
        giz = np.clip(giz, 0, nz - 1)

        RG = np.asarray(Rarr)[gix, giz]
        ZG = np.asarray(Zarr)[gix, giz]

        d_BG = np.sqrt((RG - BB[:, 0])**2 + (ZG - BB[:, 1])**2)
        d_BI = np.sqrt((BI[:, 0] - BB[:, 0])**2 + (BI[:, 1] - BB[:, 1])**2)
        ratio = d_BG / np.where(d_BI > 0, d_BI, np.nan)

        out.update({
            "eq_max_abs_ratio_minus_1": float(np.nanmax(np.abs(ratio - 1.0))),
            "eq_mean_abs_ratio_minus_1": float(np.nanmean(np.abs(ratio - 1.0))),
        })

    if mask is not None:
        # Fraction of image-point bilinear stencils fully within "valid" mask
        valid = np.asarray(mask).astype(bool)
        iL = np.searchsorted(R1d, BI[:, 0], side="right") - 1
        kL = np.searchsorted(Z1d, BI[:, 1], side="right") - 1
        iL = np.clip(iL, 0, len(R1d) - 2)
        kL = np.clip(kL, 0, len(Z1d) - 2)
        stencil_ok = (
            valid[iL,   kL] &
            valid[iL+1, kL] &
            valid[iL,   kL+1] &
            valid[iL+1, kL+1]
        )
        out["image_stencil_valid_frac"] = float(np.mean(stencil_ok))

    return out

In [ ]:
Lbs = np.array([1.76171875, 1.755859375, 1.7529296875])
Rbs = np.array([3.23828125, 3.244140625, 3.2470703125])
Bbs = np.array([-0.7610294117647058, -0.7556818181818182, -0.7528846153846154])
Tbs = -Bbs

D = 1.0
m = 1.0
n = 1.0
bc_val = 0.0
const = 1.0
timestep_for_errs_early = 10
timestep_for_errs_late = 100

w_unit = 95788333.030660808 #Omega_ci
t_unit = 1/w_unit

denses = []
dens_trues = []
masks = []
ghost_masks = []
dens_diffs = []
num_errs = []
l2_errs = []
linf_errs = []
l2_imgs = []
linf_imgs = []

lbda_errs = np.zeros_like(Lbs)
l2_bd_err = []
linf_bd_err = []
l2_inner_errs = []
l2_edges = []
mass_changes = []
mass_errs = []

for d, dat in enumerate(data):
    print("Reading data for run: ", d)
    dens = dat["Nh+"]
    src = dat["SNh+"]
    dens_true = dens.copy()

    mask = ncs[d]["in_mask"]
    #vols = ncs[d]["vol_frac"]
    ghost_id = ncs[d]["ghost_id"].values
    ghost_mask = (ghost_id >= 0).astype(bool)
    full_mask = ghost_mask | mask.astype(bool)
    #vols = np.where(mask, vols, 0)
    gst_pts = ncs[d]["ghost_pts"].values
    bdy_pts = ncs[d]["bndry_pts"].values
    img_pts = ncs[d]["image_pts"].values
    normals = ncs[d]["normals"].values
    norm_dist = ncs[d]["norm_dist"].values
    is_plasma = ncs[d]["is_plasma"].values
    Rarr = ncs[d]["R"].values
    Zarr = ncs[d]["Z"].values

    spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_early,:,:], kx=1, ky=1)
    spl_l = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_late,:,:],  kx=1, ky=1)

    nx = 64*(2**(d))
    nz = nx + 4
    dx = np.float64(1.0) / np.float64(nx)
    dz = np.float64(1.0) / np.float64(nz)

    # Centers in x; faces/nodes in z if that's your convention
    x = (np.arange(nx, dtype=np.float64) + 0.5) * dx     # centered x
    z = (np.arange(nz, dtype=np.float64)) * dz           # unshifted z

    ghosts_lo_x = x[0]  - dx*np.arange(2, 0, -1)
    ghosts_hi_x = x[-1] + dx*np.arange(1, 2+1)
    x = np.concatenate((ghosts_lo_x, x, ghosts_hi_x))
    xx,zz = np.meshgrid(x,z,indexing='ij')

    Lb, Rb, Bb, Tb = Lbs[d], Rbs[d], Bbs[d], Tbs[d]

    #Use physical grid values to map R,Z boundary to x,z (i.e. ignore ghosts)
    R_min, R_max = ncs[d]["R"].values[2,0], ncs[d]["R"].values[-3,0]
    Z_min, Z_max = ncs[d]["Z"].values[0,0], ncs[d]["Z"].values[0,-1]
    dR = ncs[d]["R"].values[1,0] - ncs[d]["R"].values[0,0]
    dZ = ncs[d]["Z"].values[0,1] - ncs[d]["Z"].values[0,0]
    xL = x[2] + (Lb - R_min) * (x[-3] - x[2]) / (R_max - R_min)
    xR = x[2] + (Rb - R_min) * (x[-3] - x[2]) / (R_max - R_min)
    zB = z[0] + (Bb - Z_min) * (z[-1] - z[0]) / (Z_max - Z_min)
    zT = z[0] + (Tb - Z_min) * (z[-1] - z[0]) / (Z_max - Z_min)

    Lx = xR - xL
    Lz = zT - zB
    xi   = (x - xL)/ Lx
    zeta = (z - zB)/ Lz
    lbda = (m*np.pi/(Rb-Lb))**2 + (n*np.pi/(Tb-Bb))**2

    xxi, zzeta = np.meshgrid(xi,zeta,indexing='ij',copy=False)
    t64 = np.asarray(dat["t_array"].values, dtype=np.float64) # 1D time
    t_norm = t64*t_unit

    print("left, right: ", xL, zB)
    print("Box size: ", Lx, Lz)

    #Compare simulation lambda to analytic value:
    #Use density near the center, which is always 1...
    n0 = dens.values[0,(nx+4)//2-1,nz//2-1]
    n1 = dens.values[1,(nx+4)//2-1,nz//2-1]
    dt = t_norm[1] - t_norm[0]
    ratio = (n1 - const) / (n0 - const)
    lambda_eff = -np.log(ratio) / (D * dt)
    #print("Effective lambda from simulation after one timestep:", lambda_eff)
    #print("Analytic lambda:", lbda)
    lbda_errs[d] = 100*np.abs((lambda_eff - lbda))/lbda
    #print("Percent error:", lbda_errs[d])

    spatial = np.cos(m * np.pi * xxi) * np.cos(n * np.pi * zzeta)
    temp = np.exp(-D * lbda * t_norm)

    dens_true[:,:,:] = const + np.asarray(spatial)[np.newaxis,:,:]*np.asarray(temp)[:,np.newaxis,np.newaxis]

    denses.append(dens)
    dens_trues.append(dens_true)
    masks.append(mask)
    ghost_masks.append(ghost_mask)

    #l2_err_b, linf_err_b = interpolate_on_boundary(
    #    Rarr[:,0], Zarr[0,:], dens, dens_true, bdy_pts[:,0], bdy_pts[:,1], method="linear",
    #    t_index = timestep_for_errs_early)
    metrics = ib_boundary_errors_from_ghostid(
        dens_t=dens.values[1, :, :],
        dens_true_t = dens_true.values[1,:,:],
        ghost_id=ghost_id,
        gst_pts=gst_pts,
        bdy_pts=bdy_pts,
        img_pts=img_pts,
        normals=normals,
        norm_dist=norm_dist,
        Rarr=Rarr,
        Zarr=Zarr,
        is_plasma=is_plasma,
        mask=full_mask,   # optional
        bc_val_dir=bc_val,
        bc_val_neu=bc_val)
    l2_err_b, linf_err_b = metrics["neu_l2"], metrics["neu_linf"]
    l2_img, linf_img = metrics["img_l2"], metrics["img_linf"]
    #print(metrics["image_stencil_valid_frac"])
    l2_bd_err.append(l2_err_b)
    linf_bd_err.append(linf_err_b)
    l2_imgs.append(l2_img)
    linf_imgs.append(linf_img)

    M0 = (dens.where(mask)*dR*dZ)[0,:,:].sum().values
    M1 = (dens.where(mask)*dR*dZ)[10,:,:].sum().values
    M2 = (dens.where(mask)*dR*dZ)[100,:,:].sum().values
    M3 = (dens.where(mask)*dR*dZ)[-1,:,:].sum().values
    MT0 = (dens_true.where(mask)*dR*dZ)[0,:,:].sum().values
    MT1 = (dens_true.where(mask)*dR*dZ)[10,:,:].sum().values
    MT2 = (dens_true.where(mask)*dR*dZ)[100,:,:].sum().values
    MT3 = (dens_true.where(mask)*dR*dZ)[-1,:,:].sum().values
    mass_errs.append(np.abs(M3-MT3))

    #Get L2 error
    N_valid = np.sum(mask)
    dens_diff = np.abs(dens.where(mask)-dens_true.where(mask))
    linf_err_t = np.max(dens_diff)
    l2_err_t = np.sqrt(np.nanmean(dens_diff**2, axis=(1, 2)))
    #l2_err_t = np.sqrt(np.sum((dens_diff**2), axis=(1, 2)) / N_valid)
    dens_diffs.append(dens_diff)
    l2_errs.append(l2_err_t)
    linf_errs.append(linf_err_t.values)

    init_dens_diff = dens_diff[0,:,:]
    mask2 = init_dens_diff.notnull() & (init_dens_diff >= 1e-10)
    nerrs = int(mask2.sum().compute())
    num_errs.append(nerrs)
    
    #V = vols*dR*dZ?
    L2_edge, edge_mask, l2_inner = l2_error_near_ghost_cells(dens, dens_true, mask, ghost_mask, V=dR*dZ, t_index=1)
    l2_edges.append(L2_edge)
    l2_inner_errs.append(l2_inner)


print('\n')
print("Eigenmode rel. err.: ", lbda_errs[0], lbda_errs[1], lbda_errs[2]) #, lbda_errs[3])
print("Eigenmode rel. err. convergence: ", lbda_errs[0]/lbda_errs[1], lbda_errs[1]/lbda_errs[2]) #, lbda_errs[2]/lbda_errs[3])

print('\n')
print("Mass change: ", mass_errs[0], mass_errs[1], mass_errs[2]) #, mass_errs[3])
print("Mass change convergence: ", mass_errs[0]/mass_errs[1], mass_errs[1]/mass_errs[2]) #, mass_errs[2]/mass_errs[3])

print('\n')
print("L2 Bdy Err: ", l2_bd_err[0], l2_bd_err[1], l2_bd_err[2]) #, l2_bd_err[3])
print("L2 Bdy Err convergence: ", l2_bd_err[0]/l2_bd_err[1], l2_bd_err[1]/l2_bd_err[2]) #, l2_bd_err[2]/l2_bd_err[3])

print('\n')
print("Linf Bdy Err: ", linf_bd_err[0], linf_bd_err[1], linf_bd_err[2]) #, linf_bd_err[3])
print("Linf Bdy Err. convergence: ", linf_bd_err[0]/linf_bd_err[1], linf_bd_err[1]/linf_bd_err[2]) #, linf_bd_err[2]/linf_bd_err[3])

print('\n')
print("L2 img error: ", l2_imgs[0], l2_imgs[1], l2_imgs[2]) #, l2_imgs[3])
print("L2 img err convergence: ", l2_imgs[0]/l2_imgs[1], l2_imgs[1]/l2_imgs[2]) #, l2_imgs[2]/l2_imgs[3])

print('\n')
print("L2 early innermost error: ", l2_inner_errs[0], l2_inner_errs[1], l2_inner_errs[2]) #, l2_inner_errs[3])
print("L2 convergence in early innermost cells: ", l2_inner_errs[0]/l2_inner_errs[1], l2_inner_errs[1]/l2_inner_errs[2]) #, l2_inner_errs[2]/l2_inner_errs[3])

print('\n')
print("L2 relative error in fluid cells touching ghosts: ", l2_edges[0], l2_edges[1], l2_edges[2]) #, l2_edges[3])
print("L2 convergence in fluid cells touching ghosts: ", l2_edges[0]/l2_edges[1], l2_edges[1]/l2_edges[2]) #, l2_edges[2]/l2_edges[3])

print('\n')
print("Cells different than analytic initial value: ", num_errs[0], num_errs[1], num_errs[2]) #, num_errs[3])

print('\n')
print("Linf errs: ", linf_errs[0], linf_errs[1], linf_errs[2]) #, linf_errs[3])
print("Linf convergence: ", linf_errs[0]/linf_errs[1], linf_errs[1]/linf_errs[2]) #, linf_errs[2]/linf_errs[3])

print('\n')
print("L2 max errs: ", np.max(l2_errs[0]), np.max(l2_errs[1]), np.max(l2_errs[2])) #, np.max(l2_errs[3]))
print("L2 early errs: ", l2_errs[0][timestep_for_errs_early], l2_errs[1][timestep_for_errs_early], l2_errs[2][timestep_for_errs_early]) #,
      #l2_errs[3][timestep_for_errs_early])
print("L2 late errs: ", l2_errs[0][timestep_for_errs_late], l2_errs[1][timestep_for_errs_late], l2_errs[2][timestep_for_errs_late]) #,
      #l2_errs[3][timestep_for_errs_late])
print("L2 final errs: ", l2_errs[0][-1], l2_errs[1][-1], l2_errs[2][-1]) #, l2_errs[3][-1])
print("L2 max convergence: ", np.max(l2_errs[0])/np.max(l2_errs[1]), np.max(l2_errs[1])/np.max(l2_errs[2])) #, np.max(l2_errs[2])/np.max(l2_errs[3]))


Reading data for run:  0


left, right:  0.25390625 0.2463235294117647
Box size:  0.4921875 0.5073529411764703


/tmp/ipykernel_566168/2468615410.py:108: RuntimeWarning: Mean of empty slice
  l2 = np.sqrt(np.nanmean(e**2))


Reading data for run:  1
left, right:  0.251953125 0.24810606060606058
Box size:  0.49609375 0.5037878787878788


/tmp/ipykernel_566168/2468615410.py:108: RuntimeWarning: Mean of empty slice
  l2 = np.sqrt(np.nanmean(e**2))


Reading data for run:  2
left, right:  0.2509765625 0.24903846153846154
Box size:  0.498046875 0.5019230769230768


/tmp/ipykernel_566168/2468615410.py:108: RuntimeWarning: Mean of empty slice
  l2 = np.sqrt(np.nanmean(e**2))




Eigenmode rel. err.:  0.0761878246040632 0.01951085311280627 0.004933416356388188
Eigenmode rel. err. convergence:  3.9048945816754714 3.954836102073978


Mass change:  0.0 4.3520742565306136e-14 3.2107649872159527e-13
Mass change convergence:  0.0 0.1355463347164592


L2 Bdy Err:  1.1929586163078527e-15 9.917592864861598e-16 4.492165986487473e-15
L2 Bdy Err convergence:  1.202871132706556 0.22077529847948452


Linf Bdy Err:  3.3553406966448843e-15 6.513308411134194e-15 1.2829243840112966e-14
Linf Bdy Err. convergence:  0.5151515151515146 0.5076923076923013


L2 img error:  0.00032006934789734047 0.00017137280372853678 8.850287583155767e-05
L2 img err convergence:  1.8676787736071971 1.9363529390242706


L2 early innermost error:  7.809746300601652e-07 1.5757373713330253e-07 3.390886628524813e-08
L2 convergence in early innermost cells:  4.956248701517339 4.646977454443947


L2 relative error in fluid cells touching ghosts:  0.0009607102144607402 0.0005249880257163374 0.0002735543308